In [9]:
import pandas as pd
import unicodedata
from datetime import date, timedelta, datetime
import logging

logging.basicConfig(level=logging.INFO)

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
pd.options.display.float_format = None

In [174]:
wager_amount = 100


def calculate_return(row, column_name, wager_amount=100):
    odds = row[column_name]
    win = row["win"]

    if win == True:
        if odds >= 100:
            return round(odds / 100 * wager_amount, 2)
        else:
            return round(wager_amount / abs(odds) * 100, 2)
    else:
        return -wager_amount


def load_data(file_path):
    try:
        return pd.read_csv(file_path)
    except FileNotFoundError:
        logging.error(f"File not found: {file_path}")
        return None

def normalize_name(name):
    normalized_name = unicodedata.normalize('NFKD', name).encode('ASCII', 'ignore').decode('utf-8')
    return normalized_name


In [3]:
sp_log = load_data('output/sp_log_2023.csv')
# sp_log = load_data('/Users/patrick/Documents/Sports Analytics/sp-k-prediction-app/output/sp_log_2023.csv')
spk_history = load_data('output/spk_history.csv')
# spk_history = load_data('/Users/patrick/Documents/Sports Analytics/sp-k-prediction-app/output/spk_history.csv')

# Date manipulation
today = date.today()
yesterday = today - timedelta(days=3)

In [190]:
start_date = datetime(2023, 5, 10).date()
end_date = datetime(2023, 8, 20).date()

# Calculate the number of days between the start and end dates
num_days = (end_date - start_date).days + 1

# Generate the array of date strings
# date_array = [(start_date + timedelta(days=i)).strftime("%Y-%m-%d") for i in range(num_days)]
date_array = [start_date + timedelta(days=i) for i in range(num_days)]

# print(date_array)

[datetime.date(2023, 5, 10), datetime.date(2023, 5, 11), datetime.date(2023, 5, 12), datetime.date(2023, 5, 13), datetime.date(2023, 5, 14), datetime.date(2023, 5, 15), datetime.date(2023, 5, 16), datetime.date(2023, 5, 17), datetime.date(2023, 5, 18), datetime.date(2023, 5, 19), datetime.date(2023, 5, 20), datetime.date(2023, 5, 21), datetime.date(2023, 5, 22), datetime.date(2023, 5, 23), datetime.date(2023, 5, 24), datetime.date(2023, 5, 25), datetime.date(2023, 5, 26), datetime.date(2023, 5, 27), datetime.date(2023, 5, 28), datetime.date(2023, 5, 29), datetime.date(2023, 5, 30), datetime.date(2023, 5, 31), datetime.date(2023, 6, 1), datetime.date(2023, 6, 2), datetime.date(2023, 6, 3), datetime.date(2023, 6, 4), datetime.date(2023, 6, 5), datetime.date(2023, 6, 6), datetime.date(2023, 6, 7), datetime.date(2023, 6, 8), datetime.date(2023, 6, 9), datetime.date(2023, 6, 10), datetime.date(2023, 6, 11), datetime.date(2023, 6, 12), datetime.date(2023, 6, 13), datetime.date(2023, 6, 14), 

In [191]:
def daily_updates(days_games):
    # output_format = "%Y-%m-%d"

    # days_games = datetime.strptime(input_string, output_format).strftime(days_games)
    sp_log['Date'] = pd.to_datetime(sp_log['Date'])
    spk_history['commence_time'] = pd.to_datetime(spk_history['commence_time'])

    spk_yesterday = spk_history[spk_history['commence_time'].dt.date == days_games]
    sp_log_yesterday = sp_log[sp_log['Date'].dt.date == days_games]

    sp_log_yesterday = sp_log_yesterday[['Name', 'SO', 'IP']]
    sp_log_yesterday['Name'] = sp_log_yesterday['Name'].apply(normalize_name)
    df = pd.merge(spk_yesterday, sp_log_yesterday, how='inner', on='Name')
    df_over = df[df['bet_over'] == 1]
    df_under = df[df['bet_under'] == 1]
    df_over['win'] = df_over['SO'] > df_over['prop_k']
    df_under['win'] = df_under['SO'] < df_under['prop_k']

    df_over['wager_return'] = df_over.apply(calculate_return, column_name='over', axis=1)
    df_under['wager_return'] = df_under.apply(calculate_return, column_name='under', axis=1)
    daily_over_return = round(df_over['wager_return'].sum(),2)
    daily_over_wager = df_over.shape[0] * wager_amount
    daily_over_roi = round(daily_over_return / daily_over_wager * 100, 2)
    daily_over_bets = df_over.shape[0]
    daily_over_losers = len(df_over[df_over['win'] == False])
    daily_over_winners = len(df_over[df_over['win'] == True])

    daily_under_return = round(df_under['wager_return'].sum(),2)
    daily_under_wager = df_under.shape[0] * wager_amount
    daily_under_roi = round(daily_under_return / daily_under_wager * 100, 2)
    daily_under_bets = df_under.shape[0]
    daily_under_losers = len(df_under[df_under['win'] == False])
    daily_under_winners = len(df_under[df_under['win'] == True])

    data = {
        "Over Bets": daily_over_bets,
        "Over Win": daily_over_winners,
        "Over Loss": daily_over_losers,
        "Over Return": daily_over_return,
        "Over ROI": daily_over_roi,
        "Under Bets": daily_under_bets,
        "Under Win": daily_under_winners,
        "Under Loss": daily_under_losers,
        "Under Return": daily_under_return,
        "Under ROI": daily_under_roi,
        "Total Bets": daily_over_bets + daily_under_bets,
        "Total Win": daily_over_winners + daily_under_winners,
        "Total Loss": daily_over_losers + daily_under_losers,
        "Total Return": daily_over_return + daily_under_return,
        "Total ROI": round((daily_over_return + daily_under_return) / (daily_over_wager + daily_under_wager) * 100, 2)
    }

    df_sum = pd.DataFrame(data, index=[days_games])
    df_sum.to_csv('output/spk_daily_summary.csv', mode='a', header=False)
    # df_sum.to_csv('output/spk_daily_summary.csv', header=True)

    df_over.to_csv('output/spk_daily_over.csv', mode='a', header=False)
    # df_over.to_csv('output/spk_daily_over.csv', header=True)
    df_under.to_csv('output/spk_daily_under.csv', mode='a', header=False)
    # df_under.to_csv('output/spk_daily_under.csv', header=True)

    return df_sum




In [192]:
for i in range(len(date_array)):
    daily_updates(date_array[i])
    # print(date_array[i])

/var/folders/y0/3zjhrtwj00scq7bwsxt8b59w0000gn/T/ipykernel_12050/931288335.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_over['win'] = df_over['SO'] > df_over['prop_k']
/var/folders/y0/3zjhrtwj00scq7bwsxt8b59w0000gn/T/ipykernel_12050/931288335.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_under['win'] = df_under['SO'] < df_under['prop_k']
/var/folders/y0/3zjhrtwj00scq7bwsxt8b59w0000gn/T/ipykernel_12050/931288335.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a s

In [181]:
date_array[0]

datetime.date(2023, 5, 9)

In [189]:
daily_updates(date_array[0])

/var/folders/y0/3zjhrtwj00scq7bwsxt8b59w0000gn/T/ipykernel_12050/2921503061.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_over['win'] = df_over['SO'] > df_over['prop_k']
/var/folders/y0/3zjhrtwj00scq7bwsxt8b59w0000gn/T/ipykernel_12050/2921503061.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_under['win'] = df_under['SO'] < df_under['prop_k']
/var/folders/y0/3zjhrtwj00scq7bwsxt8b59w0000gn/T/ipykernel_12050/2921503061.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of 

,Over Bets,Over Win,Over Loss,Over Return,Over ROI,Under Bets,Under Win,Under Loss,Under Return,Under ROI,Total Bets,Total Win,Total Loss,Total Return,Total ROI
2023-05-09,14,6,8,-292.91,-20.92,8,4,4,-76.85,-9.61,22,10,12,-369.76,-16.81


In [101]:
# spk_history

,Name,Handedness,Team,Opponent,xK,prop_k,over,over_odds,x_over,over_diff,under,under_odds,x_under,under_diff,bet_over,bet_under,commence_time
0,Michael Lorenzen,R,DET,CLE,4.11,3.5,-152.0,0.603,0.588,-0.015,120.0,0.455,0.412,-0.043,0,0,2023-05-09 18:10:00
1,Shane Bieber,R,CLE,DET,6.61,6.5,-134.0,0.573,0.490,-0.083,106.0,0.485,0.510,0.025,0,1,2023-05-09 18:10:00
2,Connor Seabold,R,COL,PIT,5.16,3.5,-126.0,0.558,0.757,0.199,-102.0,0.505,0.243,-0.262,1,0,2023-05-09 18:35:00
3,Luis Ortiz,R,PIT,COL,4.38,4.5,-146.0,0.593,0.446,-0.147,114.0,0.467,0.554,0.087,0,1,2023-05-09 18:35:00
4,Zach Eflin,R,TBR,BAL,5.15,4.5,116.0,0.463,0.585,0.122,-148.0,0.597,0.415,-0.182,1,0,2023-05-09 18:35:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2118,Bailey Ober,R,MIN,MIL,6.46,5.5,126.0,0.442,0.626,0.184,-160.0,0.615,0.374,-0.241,1,0,2023-08-22 20:10:00
2119,Graham Ashcraft,R,CIN,LAA,4.11,4.5,-164.0,0.621,0.392,-0.229,128.0,0.439,0.608,0.169,0,1,2023-08-22 21:38:00
2120,Lucas Giolito,R,LAA,CIN,6.42,6.5,-156.0,0.609,0.461,-0.148,122.0,0.450,0.539,0.089,0,1,2023-08-22 21:38:00
2121,Jon Gray,R,TEX,ARI,5.16,4.5,-142.0,0.587,0.588,0.001,112.0,0.472,0.412,-0.060,0,0,2023-08-22 21:40:00
